# Solving the 0/1 knapsack problem using the simulated annealing algorithm

#### The Knapsack Problem is a classic combinatorial optimization problem where the objective is to maximize the total value of items placed in a knapsack without exceeding its capacity. To tackle this problem, I used the simulated annealing algorithm to find optimal solutions within a reasonable time frame.

Problem Representation

1. Search state space representation - Each state is represented as a binary vector of length n (n=number of candidate items). A  value of indicates that the item is present in the state space, whereas a value of 0 indicates that the item is not present in the state space. 
2. Evaluation function - The calc_energy function evaluates the "Fitness" of a given solution. 
3. Goal - To maximise the value of items in a knapsack without exceeeding its capacity. This is done by evaluating the energy of a solution (Fitness) using the evaluation function.
4. Successor function - The negihbors function acts as the the successor function by providing the neighbor solutions of a particular solution.

Main Steps of Solution Approach

1. A data base of items, with their names, values, and weights is simulated by creating a dictionary.
2. Input of user is taken to to create seperate lists of items required, their values, and their weights.
3. An initial solution is generated using those items.
4. It is passed into the Simulated Annealing algorithm.
5. The algorithm provides the most optimal solution as a binary vector and its energy.

Main Steps of the Simulated Annealing algorithm

1. Starts with a random initial solution with some arbitrary number of energy.
2. Loops indefinitely until the temeperature is zero or there are no neighbors to the current solution. 
3. Within the loop, the energy difference between the randomly selected neighbor solution and the current solution is computed and if it is less than zero, the neighbor is set as the current solution, if it is greater than zero, the neighbor is set as the current solution if and only if the probability of accepting a worse solution is greater than  some random number between 1 and zero. 

In [1]:
import numpy as np #for efficient mathematical manipulations
import random  #to generate random numbers

In [2]:
def simulated_annealing(initial_solution, energy_func, neighbors_func, schedule_func, prob_func, values, weights, capacity, max_iterations=10000):
    """Returns an optimal solution with minimum energy and the corresponding energy

    Args:
        initial_solution (list): binary vector of items
        energy_func (function): calculates the energy of the current solution
        neighbors_func (function): returns the neighbors of the current solution
        schedule_func (function): returns a lambda function that gives the current temperature
        prob_func (function): returns a boolean depending on the probability of accepting a worse solution and the randomly generated probability
        values (list): contains values of each item
        weights (list): contains weights of each item
        capacity (int): capacirty of the knapsack
        max_iterations (int): The maximum number of iterations to run. Defaults to 10000.

    Returns:
        list, int: A binary vector of the optimal solution, energy of the optimal solution
    """
    current = initial_solution

    for t in range(max_iterations): #termination condition 1
        temperature = schedule_func()(t) #returns the temperature of the current iteration (t in this case)
        if temperature == 0:  #termination condition 2
            break 
        neighbors = neighbors_func(current, capacity, weights) #generate neighbors of the current solution

        if not neighbors: #termination condition 3
            break
        
        next = random.choice(neighbors) #randomly select a neighbor from the neighbors list
        #calc energy difference between current and neighbor solution
        delta_e = energy_func(next, values, weights, capacity) - energy_func(current, values, weights, capacity) 

        p = np.exp(-delta_e/temperature) #the probability of accepting a worse solution
        if delta_e < 0:  #accepting a better solution
            current = next
        if delta_e >= 0 and prob_func(p):  #accepting a worse solution
            current = next

    return current, energy_func(current, values, weights, capacity)

In [3]:
#This is the evaluation function
#Maximising value: The total value of the items is calculated and since this is a minimising algorithm, the value is negated to maximise it. 
#The penalty returns a postive value if the total weight is greater than the capacity. 
#Then the final energy is returned by adding the negative value of total value to the penalty. 

def calc_energy(solution, values, weights, capacity):   #a solution looks like [0,1,1,0,1,0], len(solution)=len(values)=len(weights)
    """Returns the energy of current solution

    Args:
        solution (list): binary vector of items
        values (list): corresponding values of items
        weights (list): corresponding weights of items
        capacity (int): capacity of the knapsack

    Returns:
        int: The energy of current solution
    """
    total_value = np.dot(values, solution)
    total_weight = np.dot(weights, solution)
    penalty = max(0, total_weight - capacity)**2 #ensures that the penalty will be zero if the total weight is less than the capacity (sqaured to increase the impact)

    #The aim is to maximise the total value (Increase energy_value negatively), but since SA is a minimising algorithm, taking the negative of total_value solves this. 
    #The penalty is added to the total_value to discourage the particular solution by increasing the energy_value (Increase energy_value positively)
    energy_value = -total_value + penalty  #need a solution with high negative value (i.e max of ||total value||)
    return energy_value

In [4]:
def schedule(init_temp=30, cooling_rate=0.98, limit=10000):
    """Returns a lambda function of time (iteration) to calculate the temperature

    Args:
        init_temp (int): Initial temperature. Defaults to 30.
        cooling_rate (float): Rate at which the temperature decreases. Defaults to 0.98.
        limit (int): Limit of iterations. Defaults to 10000.

    Returns:
        lambda function: Calculates the temperature of the current iteration
    """

    return lambda t: init_temp*(np.exp(-cooling_rate*t)) if t < limit else 0

In [5]:
def neighbors(solution, capacity, weights):
    """Returns a list of neighbors of the current solution

    Args:
        solution (list): A binary vector of items
        capacity (int): The capacity of the knapsack
        weights (list): A list of weights of the items

    Returns:
        list: List of neighbors of the current solution
    """
    #By flipping a bit in the current solution
    neighbors = []
    for i in range(len(solution)):
        neighbor = solution.copy() #to avoid modifying the original 'solution'
        neighbor[i] = 1 - neighbor[i] #flip the bit
        total_weight = np.dot(neighbor, weights)
        if total_weight <= capacity:
            neighbors.append(neighbor)

    return neighbors

In [6]:
def probability(p):
    """Return true if p is greater than a random number between 0 and 1

    Args:
        p (float): The probability of accepting a worse solution

    Returns:
        boolean: True if p is greater than a random number between 0 and 1
    """
    return p > random.uniform(0.0,1.0)

In [7]:
def init_solution(items, capacity, weights, max_attempts=20):
    """Generates a random binary vector of length = length of items

    Args:
        items (list): A list of items to consider
        capacity (int): Capacity of the knapsack
        weights (list): A list of weights of the items
        max_attempts (int, optional): Maximum allowed iterations to find a feasible solution. Defaults to 20.

    Raises:
        ValueError: If a feasible solution with given constraints is not found in the maximum number of attempts

    Returns:
        list: A binary vector representation of the initial solution
    """

    #A for loop within a maximum attempt range is necessary instead of a forever loop to increase efficiency,
    #and if the problem is large, then generating many solutions till a feasible one is found is computationally expensive
    for attempt in range(max_attempts):
        solution = [random.choice([0,1]) for _ in range(len(items))]
        total_weight = np.dot(solution, weights)

        if total_weight <= capacity:
            print(f"Attempt {attempt+1}: Initial solution is feasible. Since, Weight = {total_weight} <= Capacity = {capacity}")
            return solution
        else:
            print(f"Attempt {attempt+1}: Initial solution is not feasible. Since, Weight = {total_weight} > Capacity = {capacity}")
    raise ValueError("Failed to generate a feasible solution within the maximum number of attempts ):")

In [8]:
def all_items_dict():  
    """Acts as a database of items

    Returns:
        dictionary: A dictionary of items with corresponding values and weights
    """
    #1-Phone, 2-Laptop, 3-Tablet, 4-Camera, 5-Headphones, 6-Keyboard, 7-Mouse
    #8-Gaming Console, 9-Portable Charger, 10-Water Bottle, 11-Book, 12-Umbrella
    
    items_dict = {
        '1': {
            'name': 'Phone',
            'value': 150,
            'weight': 7
        },

        '2': {
            'name': 'Laptop',
            'value': 80,
            'weight': 30
        },

        '3': {
            'name': 'Tablet',
            'value': 20,
            'weight': 18
        },

        '4': {
            'name': 'Camera',
            'value': 40,
            'weight': 60
        },

        '5': {
            'name': 'Headphones',
            'value': 50,
            'weight': 12
        },

        '6': {
            'name': 'Keyboard',
            'value': 20,
            'weight': 20
        },

        '7': {
            'name': 'Mouse',
            'value': 30,
            'weight': 8
        },

        '8': {
            'name': 'Gaming Console',
            'value': 120,
            'weight': 40
        }, 

        '9': {
            'name': 'Portable Charger',
            'value': 60,
            'weight': 10
        }, 

        '10': {
            'name': 'Water Bottle',
            'value': 30,
            'weight': 5
        },

        '11': {
            'name': 'Book',
            'value': 10,
            'weight': 4
        },

        '12': {
            'name': 'Umbrella',
            'value': 20,
            'weight': 15
        }
    }

    return items_dict

In [9]:
def get_user_input(prompt, num_items):
    """Gets user input

    Args:
        prompt (string): Question to get user input
        num_items (_type_): Total number of items

    Returns:
        string: A comma separated string of user input intergers
    """
    while True: #loops until a correct input is entered
        try:
            user_input = input(prompt) #get user input
            user_input = user_input.replace(" ", "")  #remove spaces
            user_input_list = user_input.split(",") #split elements into a list
            
            #check if all inputs are digits and <= num_items
            if all(item.isdigit() and (1 <= int(item) <=num_items) for item in user_input_list):
                return user_input
            else:
                print(f"Enter 'NUMBERS' between 1 and {num_items}.")

        except ValueError:
            print('Invalid input!')
            continue

In [10]:
def gen_items(prompt, all_items_dict):
    """Generates items requested by the user with their values and weights

    Args:
        prompt (string): The prompt to get user input
        all_items_dict (dictionary): A dictionary contaning all available items

    Returns:
        list, list, list: Lists of user requested items, values, and weights
    """
    user_input = get_user_input(prompt, len(all_items_dict)) #this a string like '1','4','3'
    input_items = user_input.split(',') #this is a list like ['1', '4', '3']

    user_items, values, weights = [],[],[]
    
    #iterates over the available items dictionary and appends the items requested by the user and their corresponding values and weights to respective lists
    for key in all_items_dict:  
        if key in input_items:
            user_items.append(all_items_dict[key]['name'])
            values.append(all_items_dict[key]['value'])
            weights.append(all_items_dict[key]['weight'])

    return user_items, values, weights


In [11]:
def get_solution_items(final_solution, user_items):
    """Returns the list of items in the final solution

    Args:
        final_solution (list): A binary vector of the final solution
        user_items (list): A list of user requested items

    Returns:
        list: List of items in the final solution
    """
    final_solution_items = []

    for i in range(len(final_solution)):
        if final_solution[i] == 1:  #1 represents if the item is in the final solution
            final_solution_items.append(user_items[i]) 
    
    return final_solution_items

In [12]:
def main():
    """Run the program"""
    prompt = ("Enter the key number of the item you want to put in your bag.\n"
          "1 : Phone\n"
          "2 : Laptop\n"
          "3 : Tablet\n"
          "4 : Camera\n"
          "5 : Headphones\n"
          "6 : Keyboard\n"
          "7 : Mouse\n"
          "8 : Gaming Console\n"
          "9 : Portable Charger\n"
          "10 : Water Bottle\n"
          "11 : Book\n"
          "12 : Umbrella")
    
    items_db = all_items_dict() #dictionary of available items
    user_req_items, values, weights = gen_items(prompt, items_db) #user requested items and their values and weights

    knapsack_capacity = 50 #the capacity of the knapsack (max weight)
    initial_solution = init_solution(user_req_items, knapsack_capacity, weights) #an initial solution made using the user requested items

    result, energy = simulated_annealing(initial_solution, calc_energy, neighbors, schedule, probability, values, weights, knapsack_capacity)
    print("------Final Result------")
    print(f"Optimal solution: {result}\nEnergy: {energy}")
    print(f"The items requested: {user_req_items}")
    print(f"The items you can take are: {get_solution_items(result, user_req_items)}")

In [13]:
main()

Attempt 1: Initial solution is not feasible. Since, Weight = 55 > Capacity = 50
Attempt 2: Initial solution is not feasible. Since, Weight = 68 > Capacity = 50
Attempt 3: Initial solution is feasible. Since, Weight = 37 <= Capacity = 50
------Final Result------
Optimal solution: [1, 1, 0, 0, 1, 0]
Energy: -290
The items requested: ['Phone', 'Laptop', 'Tablet', 'Gaming Console', 'Portable Charger', 'Water Bottle']
The items you can take are: ['Phone', 'Laptop', 'Portable Charger']


C:\Users\HP\AppData\Local\Temp\ipykernel_19100\1867588337.py:33: RuntimeWarning: overflow encountered in scalar divide
  p = np.exp(-delta_e/temperature) #the probability of accepting a worse solution
